In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MaxDelta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MaxDelta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MaxDelta\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MaxDelta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df_steamreview = pd.read_excel("/content/Data_Ulasan_Video_Game_Lokal.xlsx")

In [ ]:
df_steamreview.head()

,Steam ID,Profile URL,Review Text,Review,Review Length,Play Hours,Data Posted,Nama Video Game,App ID
0,harshlycritical,https://steamcommunity.com/id/harshlycritical/,I haven't felt compelled to write a review in ...,Positive,944,5.8 hrs on record,"Posted: 24 February, 2015",Dreadout,269790
1,EverchosenofChaos,https://steamcommunity.com/id/EverchosenofChaos/,I am very sattisfied with this experience. Sho...,Positive,711,6.3 hrs on record,"Posted: 3 June, 2017",Dreadout,269790
2,VirtualVictory,https://steamcommunity.com/id/VirtualVictory/,Although it may not be one of the most graphic...,Positive,880,40.9 hrs on record,"Posted: 19 August, 2015",Dreadout,269790
3,AestheticGamer,https://steamcommunity.com/id/AestheticGamer/,DreadOut is an Indonesian survivalhorror game ...,Positive,6295,21.7 hrs on record,"Posted: 8 May, 2014",Dreadout,269790
4,markdj,https://steamcommunity.com/id/markdj/,Scary\nAtmosphere\nClever 'Game over' sequence...,Positive,145,4.2 hrs on record,"Posted: 8 October, 2014",Dreadout,269790


In [ ]:
def preprocess(text):
    text = text.lower()
    text = text.strip()
    text = re.compile('<.*?>').sub(' ', text)
    text = re.compile('[%s]'% re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+',' ', text)
    text = re.sub(r'\[[0-9]*\]',' ', text)
    text = re.sub(r'[^\w\s]',' ',str(text).lower().strip())
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    return text

In [ ]:
#stopword removal english
def stopword(string):
    a = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [ ]:
#Lemmatizer
wl = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get Position tags
    a = [wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)]
    return " ".join(a)

In [ ]:
def finalpreprocess(string):
    string = str(string)
    return lemmatizer(stopword(preprocess(string)))
df_steamreview['Review Text'] = df_steamreview['Review Text'].apply(lambda x: finalpreprocess(x))
df_steamreview.head(1000)

,Steam ID,Profile URL,Review Text,Review,Review Length,Play Hours,Data Posted,Nama Video Game,App ID
0,harshlycritical,https://steamcommunity.com/id/harshlycritical/,felt compel write review think game deserve on...,Positive,944,5.8 hrs on record,"Posted: 24 February, 2015",Dreadout,269790
1,EverchosenofChaos,https://steamcommunity.com/id/EverchosenofChaos/,sattisfied experience short awesomely great be...,Positive,711,6.3 hrs on record,"Posted: 3 June, 2017",Dreadout,269790
2,VirtualVictory,https://steamcommunity.com/id/VirtualVictory/,although may one graphically appeal survival h...,Positive,880,40.9 hrs on record,"Posted: 19 August, 2015",Dreadout,269790
3,AestheticGamer,https://steamcommunity.com/id/AestheticGamer/,dreadout indonesian survivalhorror game take i...,Positive,6295,21.7 hrs on record,"Posted: 8 May, 2014",Dreadout,269790
4,markdj,https://steamcommunity.com/id/markdj/,scary atmosphere clever game sequence sidemiss...,Positive,145,4.2 hrs on record,"Posted: 8 October, 2014",Dreadout,269790
...,...,...,...,...,...,...,...,...,...
995,SellingSouls28,https://steamcommunity.com/id/SellingSouls28/,game gay say,Positive,62,6.0 hrs on record,"Posted: October 27, 2018",Dreadout,269790
996,gobbe,https://steamcommunity.com/id/gobbe/,npc walk like bug super slow motion walk path,Negative,68,0.2 hrs on record,"Posted: August 23, 2017",Dreadout,269790
997,big_boss_did_nothing_wrong,https://steamcommunity.com/id/big_boss_did_not...,pay money would actually consider suicide bad ...,Negative,116,0.5 hrs on record,"Posted: September 28, 2017",Dreadout,269790
998,CrazZz,https://steamcommunity.com/id/CrazZz/,good horror game,Positive,14,0.7 hrs on record,"Posted: November 24, 2016",Dreadout,269790


In [ ]:
df_steamreview.to_csv("Steamreviewpreprocessing.csv")